In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Data Cleaning

In [2]:
train=pd.train=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/5th PROJECT/Property_train.csv')

In [3]:
test=pd.train=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/5th PROJECT/Property_test_share.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62035 entries, 0 to 62034
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Junk                   62035 non-null  int64 
 1   InteriorsStyle         62035 non-null  object
 2   PriceIndex8            62035 non-null  object
 3   ListDate               62035 non-null  object
 4   Material               62035 non-null  object
 5   PriceIndex9            62035 non-null  int64 
 6   Agency                 62035 non-null  object
 7   AreaIncomeType         62035 non-null  object
 8   EnvRating              62035 non-null  object
 9   PriceIndex7            62035 non-null  object
 10  ExpeditedListing       62035 non-null  int64 
 11  PriceIndex4            62035 non-null  object
 12  PriceIndex1            62035 non-null  object
 13  PriceIndex6            62035 non-null  object
 14  PRIMEUNIT              62035 non-null  object
 15  Channel            

In [5]:
cat_to_numeric=['PriceIndex'+str(i) for i in range(1,10)]

In [6]:
train['data']='train'
test['data']='test'

all_data=pd.concat([train,test],0,sort=False)

In [7]:
for col in cat_to_numeric:
    all_data[col]=pd.to_numeric(all_data[col],errors='coerce')
    all_data[col]=all_data[col].fillna(all_data.loc[all_data['data']=='train',col].median())

In [8]:
cat_cols=['ListDate','InteriorsStyle','Material', 'Agency', 'AreaIncomeType',
       'EnvRating', 'PRIMEUNIT', 'Channel', 'PlotType', 'Architecture',
       'Region', 'SubModel', 'Facade', 'State', 'RegionType','Zip']

In [9]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=1000].index[:-1]
    
    for cat in cats:
        name=col+'_'+str(cat)
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [10]:
all_data.shape

(72983, 125)

In [11]:
target='Junk'

In [12]:
x_train=all_data.drop([target,'data'],1)[all_data['data']=='train']
y_train=all_data[target][all_data['data']=='train']
x_test=all_data.drop([target,'data'],1)[all_data['data']=='test']

# Model Fitting

In [13]:
clf = RandomForestClassifier()

In [14]:
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

In [15]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=10,
                                   scoring='roc_auc',
                                   cv=5,
                                  n_jobs=-1,verbose=20)

In [16]:
random_search.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': [None, 'balanced'],
                                        'criterion': ['entropy', 'gini'],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      50, 70],
                                        'max_features': [5, 10, 20, 25, 30, 35],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300, 500,
                                                         700, 1000]},
                   scoring='roc_auc', verbose=20)

In [17]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [18]:
random_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_features=20,
                       min_samples_leaf=15, n_estimators=1000)

In [19]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.76397 (std: 0.00812)
Parameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 15, 'max_features': 20, 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.76394 (std: 0.00721)
Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 20, 'max_features': 35, 'max_depth': 15, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.76305 (std: 0.00786)
Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 20, 'max_features': 25, 'max_depth': 50, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': True}

Model with rank: 4
Mean validation score: 0.76115 (std: 0.00722)
Parameters: {'n_estimators': 300, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_features': 5, 'max_depth': 50, 'criterion': 'entropy', 'class_weight': None, 'bootst

In [20]:
forest=RandomForestClassifier(**{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 20, 'max_features': 35, 'max_depth': 15, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': True}
)

In [21]:
forest.fit(x_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=15, max_features=35,
                       min_samples_leaf=20, min_samples_split=10,
                       n_estimators=300)

In [22]:
Final=pd.DataFrame({target:forest.predict(x_test)})

In [23]:
Final.to_csv('sample_submission.csv',index=False)